In [1]:
list_features = []
for i in range (0,4):
    for j in range (0,4):
        for k in range (0,4):
            for w in range (0,4):
                list_features.append([i,j,k,w])
#print (np.shape(list_features))

In [2]:
from itertools import groupby
import tensorflow as tf
import random
import torch
import numpy as np
####### READING THE FASTA CHAIN DATA FROM THE FILE ############
def fasta_iter(name):
    fh = open(name)

    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))

    for header in faiter:
        # drop the ">"
        headerStr = header.__next__()[1:].strip()

        # join all sequence lines to one.
        seq = "".join(s.strip() for s in faiter.__next__())

        yield (headerStr, seq)

data=fasta_iter("Ecoli_prom.fa.txt")
training_data1=[]
training_data2=[]


######## CONVERTING DNA SEQUENCE INTO ONE HOT ENCODING ##########

for chain in data:
    headerStr, seq = chain
    temp=[]
    for i in range (len(seq)-4):
        temp1 = []
        for j in range (0,4):
            if(seq[i+j]=='A'):
                temp1.append(0)
            if(seq[i+j]=='T'):
                temp1.append(1)
            if(seq[i+j]=='G'):
                temp1.append(2)
            if(seq[i+j]=='C'):
                temp1.append(3)
        temp.append(temp1)
    training_data1.append(temp)
       
    
    
data=fasta_iter("Ecoli_non_prom.fa.txt")
for chain in data:
    headerStr, seq = chain
    temp=[]
    for i in range (len(seq)-4):
        temp1 = []
        for j in range (0,4):
            if(seq[i+j]=='A'):
                temp1.append(0)
            if(seq[i+j]=='T'):
                temp1.append(1)
            if(seq[i+j]=='G'):
                temp1.append(2)
            if(seq[i+j]=='C'):
                temp1.append(3)
        #print (temp1)
        temp.append(temp1)
    training_data2.append(temp)
#training_data
list_freq1 = []
list_freq2 = []

for i in training_data1:
    temp = []
    for j in range (len(list_features)):
        temp.append(0)
    for j in i:
        for k in range (len(list_features)):
            if (list_features[k]==j):
                temp[k] += 1
    list_freq1.append(temp)
    
for i in training_data2:
    temp = []
    for j in range (len(list_features)):
        temp.append(0)
    for j in i:
        for k in range (len(list_features)):
            if (list_features[k]==j):
                temp[k] += 1
    list_freq2.append(temp)    

In [3]:
########### CREATING LABELS FOR THE OUTPUT ###########
labels1=[]
labels2=[]

for i in range(839):
    labels1.append(1)
for i in range(3000):
    labels2.append(0)

#### SHUFFLE THE DATA ########    

comb=list(zip(list_freq1,labels1))
random.shuffle(comb)
list_freq1,labels1=zip(*comb)

comb=list(zip(list_freq2,labels2))
random.shuffle(comb)
list_freq2,labels2=zip(*comb)

print(np.shape(list_freq1))
print(np.shape(labels1))

#training_data=np.reshape(training_data,(3839,77*16))

#print(np.shape(training_data))


(839, 256)
(839,)


In [4]:
####### SPLITTING DATA INTO TRAIN,VALIDATION AND TEST ############
X_train=list()
Y_train=list()
for i in range(649):
    X_train.append(list_freq1[i])
    Y_train.append(labels1[i])
for i in range(2480):
    X_train.append(list_freq2[i])
    Y_train.append(labels2[i])


# X_train.append(list(list_freq1[:649]))
# Y_train.append(list(labels1[:649]))
# X_train.append(list(list_freq2[:2480]))
# Y_train.append(list(labels2[:2480]))

comb=list(zip(X_train,Y_train))
random.shuffle(comb)
X_train,Y_train=zip(*comb)

X_validation=[]
Y_validation=[]

for i in range(649,720):
    X_validation.append(list_freq1[i])
    Y_validation.append(labels1[i])
for i in range(2480,2740):
    X_validation.append(list_freq2[i])
    Y_validation.append(labels2[i])
# X_validation.append(list_freq1[649:720])
# Y_validation.append(labels1[:720])
# X_validation.append(list_freq2[2480:2740])
# Y_validation.append(labels2[2480:2740])

comb=list(zip(X_validation,Y_validation))
random.shuffle(comb)
X_validation,Y_validation=zip(*comb)

X_test=[]
Y_test=[]
for i in range(720,839):
    X_test.append(list_freq1[i])
    Y_test.append(labels1[i])
for i in range(2740,3000):
    X_test.append(list_freq2[i])
    Y_test.append(labels2[i])
# X_test.append(list_freq1[720:])
# Y_test.append(labels1[720:])
# X_test.append(list_freq2[2740:])
# Y_test.append(labels2[2740:])

comb=list(zip(X_test,Y_test))
random.shuffle(comb)
X_test,Y_test=zip(*comb)

X_test=list(X_test)
X_validation=list(X_validation)
X_train=list(X_train)

Y_test=list(Y_test)
Y_validation=list(Y_validation)
Y_train=list(Y_train)


X_train = np.array(X_train)
X_valid=np.array(X_validation)
X_test=np.array(X_test)

Y_test=np.array(Y_test)
Y_valid=np.array(Y_validation)
Y_train=np.array(Y_train)

# X_train=list(X_train)
# np.shape(X_train)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier


/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
parameters={
    "booster":'gbtree',
    'learning_rate':0.03,
    "n_estimators":1200,
    "max_depth":2,
    "seed":3,
    "min_child_weight":6,
    
    "colsample_bylevel":0.9,
    "colsample_bytree":0.9,
    

}

In [7]:
model = XGBClassifier(**parameters)
model.fit(X_train, Y_train)
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=2, min_child_weight=6, missing=None, n_estimators=1200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=3, silent=True,
       subsample=1)

In [8]:
# make predictions for test data

y_pred1 = model.predict(X_train)
predictions1 = [round(value) for value in y_pred1]
y_pred = model.predict(X_valid)
predictions = [round(value) for value in y_pred]

/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
# evaluate predictions
accuracy = accuracy_score(Y_train, predictions1)
print("Train Accuracy: %.2f%%" % (accuracy * 100.0))
accuracy = accuracy_score(Y_valid, predictions)
print("Validation Accuracy: %.2f%%" % (accuracy * 100.0))

Train Accuracy: 95.81%
Validation Accuracy: 91.24%


In [10]:
y_pred2 = model.predict(X_test)
predictions2 = [round(value) for value in y_pred2]
accuracy = accuracy_score(Y_test, predictions2)
print("Test Accuracy: %.2f%%" % (accuracy * 100.0))

Test Accuracy: 89.45%


/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
